In [22]:
import pandas as pd

complaints = pd.read_csv("../data/amex_complaints_sentiment.csv")
print("Complaints loaded:", complaints.shape)


Complaints loaded: (5409, 20)


In [23]:
complaints['date_received'] = pd.to_datetime(complaints['date_received'])
monthly_volume = complaints.resample('M', on='date_received').size().reset_index(name='complaint_count')

monthly_volume.to_csv("../outputs/monthly_complaint_volume.csv", index=False)
print("Exported: monthly_complaint_volume.csv")


Exported: monthly_complaint_volume.csv


C:\Users\ASUS\AppData\Local\Temp\ipykernel_26032\3635133572.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_volume = complaints.resample('M', on='date_received').size().reset_index(name='complaint_count')


In [24]:
sentiment_issue = complaints.groupby(['issue', 'sentiment_label']).size().unstack(fill_value=0).reset_index()
sentiment_issue.to_csv("../outputs/sentiment_by_issue.csv", index=False)
print("Exported: sentiment_by_issue.csv")


Exported: sentiment_by_issue.csv


In [25]:
response_dispute = complaints.groupby(['timely_response?', 'consumer_disputed?']).size().unstack(fill_value=0).reset_index()
response_dispute.to_csv("../outputs/response_vs_dispute.csv", index=False)
print("Exported: response_vs_dispute.csv")


Exported: response_vs_dispute.csv


In [26]:
top_issues = complaints['issue'].value_counts().head(10).reset_index()
top_issues.columns = ['issue', 'count']
top_issues.to_csv("../outputs/top_10_issues.csv", index=False)
print("Exported: top_10_issues.csv")


Exported: top_10_issues.csv


In [27]:
channel_summary = complaints['submitted_via'].value_counts().reset_index()
channel_summary.columns = ['channel', 'count']
channel_summary.to_csv("../outputs/complaints_by_channel.csv", index=False)
print("Exported: complaints_by_channel.csv")


Exported: complaints_by_channel.csv


In [28]:
df = pd.read_csv("../data/intermediate data/credit_card_churn.csv")

df['churn'] = df['Attrition_Flag'].apply(lambda x: 1 if x == 'Attrited Customer' else 0)

df = df.drop(['CLIENTNUM', 'Attrition_Flag'], axis=1)

cat_cols = df.select_dtypes(include='object').columns.tolist()
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

leak_cols = [col for col in df_encoded.columns if col.startswith("Naive_Bayes_Classifier")]
df_encoded = df_encoded.drop(columns=leak_cols)

X = df_encoded.drop('churn', axis=1)
y = df_encoded['churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from joblib import load
model = load("../outputs/churn_model.pkl")

y_pred = model.predict(X_test)

churn_summary = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred
})
churn_summary.to_csv("../outputs/churn_predictions.csv", index=False)
print("Exported: churn_predictions.csv")


Exported: churn_predictions.csv
